# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

### Описание проекта


Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.


Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.


## Откройте и изучите файл

In [1]:
# Импортируем все необходимое для работы
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# Отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

# Откроем файл и выведем первые 20 строк
try:
    df = pd.read_csv('//datasets/users_behavior.csv')
except:
    df = pd.read_csv('users_behavior.csv')
    
df.head(20)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [2]:
# Посмотрим информацию по датафрейму 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [22]:
# Изменим тип данных в столбцах calls и messages на целочисленный. 
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')

In [4]:
# Посмотрим какой тариф выбирают большинство клиентов 
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

В выборке присутствует дисбаланс.

## Разбейте данные на выборки

В нашем исследовании будем использовать модуль GridSearchCV. Он используется для автоматического подбирания параметров для модулей машинного обучения. Путем обычного перебора модуль GridSearchCV находит наилучшие параметры и создает модель для каждой возможной комбинации параметров.

In [12]:
# Разделим данные. В target целевой признак, а в features поместим оставшшиеся признаки
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

Используем кросс - валидацию. Она берет обучающую выборку и делит валидационную на несколько слоёв. Делим выборку на 2 части 80% и 20%.

80% будут разделены на 4 обучающие части и 1 валидационную. 

In [13]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size = 0.2, random_state = 12345, stratify = target)

In [14]:
features_train.shape

(2571, 4)

In [15]:
features_test.shape

(643, 4)

## Исследуйте модели

### Дерево решений

Подберем гиперпараметры с помощью функции GridSearchCV 

In [17]:
# Создаём словарь и вносим гиперпараметры для DecisionTree, которые будем прогонять
# criterion gini - критерий, показывающий меру сходства наборов данных
# criterion entropy - критерий неоднородности наборов данных
# max_depth - максимальная глубина дерева решений, она будет изменяться от 1 до 15
# min_samples_split - минимальное количество образцов для сплита, оно будет изменяться от 3 до 12 с шагом 3 
# min_samples_leaf - минимальное количество образцов в листах, оно будет изменяться от 1 до 10

parametrs = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(1, 15),
    'min_samples_split' : range(3, 12, 3),
    'min_samples_leaf' : range(1, 10)
}
# Объявляем классификатор DecisionTree
tree_m = DecisionTreeClassifier(random_state=12345)

# Запускаем обучение
grid = GridSearchCV(estimator = tree_m, param_grid = parametrs, cv = 5)
grid.fit(features_train, target_train)

# Выводим accuracy и наилучшие варианты гиперпараметров 
print('Точность наилучшей модели', grid.best_score_)
print('Критерий качества разбиения на классы', grid.best_params_['criterion'])
print('Глубина дерева решений', grid.best_params_['max_depth'])
print('Колличество выборок для разделения', grid.best_params_['min_samples_split'])
print('Колличество выборок в листе', grid.best_params_['min_samples_leaf'])


Точность наилучшей модели 0.8024132069056703
Критерий качества разбиения на классы gini
Глубина дерева решений 6
Колличество выборок для разделения 3
Колличество выборок в листе 9


### Случайный лес

Подберем гиперпараметры с помощью функции GridSearchCV 

In [18]:
# Создаём словарь и вносим гиперпараметры для RandomForest, которые будем прогонять
# n_estimators - 
# criterion gini - критерий, показывающий меру сходства наборов данных
# criterion entropy - критерий неоднородности наборов данных
# max_depth - максимальная глубина дерева решений, она будет изменяться от 1 до 15
# min_samples_split - минимальное количество образцов для сплита, оно будет изменяться от 3 до 12 с шагом 3 
# min_samples_leaf - минимальное количество образцов в листах, оно будет изменяться от 1 до 10

parametrs = {
    'n_estimators': [5, 30, 5],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(1, 15),
    'min_samples_split' : range(3, 12, 3),
    'min_samples_leaf' : range(1, 10)
}

# Объявляем классификатор RandomForest
forest_m = RandomForestClassifier(random_state=12345)

# Запускаем обучение
grid = GridSearchCV(estimator = forest_m, param_grid = parametrs, cv = 5)
grid.fit(features_train, target_train)

# Выводим accuracy и наилучшие варианты гиперпараметров 
print('Точность наилучшей модели', grid.best_score_)
print('Колличество деревьев', grid.best_params_['n_estimators'])
print('Критерий качества разбиения на классы', grid.best_params_['criterion'])
print('Глубина дерева решений', grid.best_params_['max_depth'])
print('Колличество выборок для разделения', grid.best_params_['min_samples_split'])
print('Колличество выборок в листе', grid.best_params_['min_samples_leaf'])


Точность наилучшей модели 0.8101990857919988
Колличество деревьев 30
Критерий качества разбиения на классы gini
Глубина дерева решений 10
Колличество выборок для разделения 3
Колличество выборок в листе 5


### Логистическая регрессия

Подберем гиперпараметры с помощью функции GridSearchCV 

In [19]:
# Создаём словарь и вносим гиперпараметры для LogisticRegression, которые будем прогонять
# solver - параметр, который помогает выбрать алгоритм, который будет строить модель
# C - параметр отвечающий за разброс данных, чтобы модель не переобучалась
parametrs = {
    'solver' : ['lbfgs', 'liblinear', 'sag', 'newton-cg'],
    'C' : [0.01, 0.1, 1, 10]
}

# Объявляем классификатор LogisticRegression
logistic_m = LogisticRegression(penalty = 'l2', random_state = 12345, max_iter = 10000)

# Запускаем обучение
grid = GridSearchCV(estimator = logistic_m, param_grid = parametrs, cv = 5)
grid.fit(features_train, target_train)
best_model = grid.best_estimator_
accuracy_logistic_m = best_model.score(features_test, target_test)

# Выводим наилучшие варианты гиперпараметров 
print('Лучший алгоритм оптимизации:', best_model.get_params()['solver'])
print('Лучшее значение С:', best_model.get_params()['C'])
print('Доля верных прогнозов:', accuracy_logistic_m)


Лучший алгоритм оптимизации: newton-cg
Лучшее значение С: 0.01
Доля верных прогнозов: 0.7433903576982893


## Проверьте модель на тестовой выборке

In [20]:
# Создаём модель случайного леса
forest_m = RandomForestClassifier(n_estimators = 30, max_depth = 10, min_samples_split = 3,
                                  min_samples_leaf = 5, random_state=12345)
# Обучаем модель
forest_m.fit(features_train, target_train)

# Рассчитаем точнось лучшей модели случайного леса в тестовой выборке
accuracy_forest_test = forest_m.score(features_test, target_test)

print('Точность лучшей модели случайного леса в тестовой выборке:', accuracy_forest_test)

Точность лучшей модели случайного леса в тестовой выборке: 0.8164852255054432


## (бонус) Проверьте модели на адекватность

Проверим нашу модель с помощью DummyClassifier. Сравним как можель обучилась на наших данных с тем, как бы она вела себя без обучения. Если показатели Dummy будут ниже, чем у нашей лучшей обученной модели, значит мы достигли поставленной задачи.

In [23]:
strategy = ['most_frequent', 'stratified', 'prior', 'uniform']

for i in strategy:
    dmc = DummyClassifier(strategy = i, random_state = 12345)
    score = dmc.fit(features_train, target_train).score(features_test, target_test)
    print('Доля верных прогнозов ложной модели при strategy =', i, score)

print('_' * 100)   
print('Доля верных прогнозов наилучшей модели случайного леса в тестовой выборке:', accuracy_forest_test)

Доля верных прогнозов ложной модели при strategy = most_frequent 0.6936236391912908
Доля верных прогнозов ложной модели при strategy = stratified 0.578538102643857
Доля верных прогнозов ложной модели при strategy = prior 0.6936236391912908
Доля верных прогнозов ложной модели при strategy = uniform 0.4821150855365474
____________________________________________________________________________________________________
Доля верных прогнозов наилучшей модели случайного леса в тестовой выборке: 0.8164852255054432


### Итоги выполненной работы:

1. Мы изучили данные чтобы разделить их на выборке, а так же обнаружили дисбаланс в соотношении.

2. Использовали модуль GridSearchCV для автоматического подбирания параметров для модулей машинного обучения. Применили кросс - валидацию. Она берет обучающую выборку и делит валидационную на несколько слоёв. Разделили выборку на 2 части: 80% и 20%.

3. Исследовали 3 модели: дерево решений, случайный лес и логистическую регрессию. Сделали вывод, что лучшие результаты у наилучшей модели случайного леса: точность наилучшей модели 0.8101990857919988.

4. Проверили наилучшую модель случайного леса на тестовой выборке и получили точность 0.8164852255054432

5. Выполнили проверку наилучшей модели случайного леса на адекватность. Доля верных прогнозов для нашей модели значительно выше прогнозов ложной модели, следовательно наше модель работает правильно и соответсвует требования заказчика.

### Вывод
Наиболее адекватные предсказания делает модель Random Forest (случайный лес). Это подтверждается точностью предсказаний на валидационной и тестовой выборке.